In [2]:
from collections import defaultdict
from itertools import product
from typing import Tuple
from functools import reduce
import pandas as pd
import numpy as np
from math import log2
import random

In [ ]:
# parses a benchfile into a Python dictionary and list of input wires
def createDict(filename: str) -> Tuple[dict, list]:
   benchfile = open(filename)
   # parsing benchfile into dictionary
   # {key : value} = {output_wire : [gate, input1, input2, ... , inputN]}
   bench = benchfile.read().splitlines()
   bench = [line for line in bench if '#' not in line and '' != line]
   inputs = [line for line in bench if 'INPUT' in line]
   circuit = [line for line in bench if (
       line not in inputs) and 'OUTPUT' not in line]

   gateOut = [line.split(' = ')[0] for line in circuit]
   gate = [line.split(' = ')[1] for line in circuit]
   gate = [[line.split('(')[0]] + line.split('(')
           [1].replace(')', '').split(', ') for line in gate]

   inputs = [line.split('(')[1].replace(')', '') for line in inputs]
   circ = {gateOut[i]: line for i, line in enumerate(gate)}

   return circ, inputs


In [ ]:
# simulates an input combination given a circuit dictionary and list of input wires  
def sim(input_comb: list, circ: dict, inputs: list) -> dict:
   netlist = {i: -1 for i in (inputs + list(circ.keys()))}
   for i, val in enumerate(input_comb):
      netlist[inputs[i]] = val

   for wire in netlist:
      if netlist[wire] == -1:
         visited = set()
         netlist[wire] = simDFS(circ, wire, netlist, visited)
   return netlist # outputs a truth table line with all input, net, and output wires
   

# recursive function for finding the value of each wire 
def simDFS(circ: dict, wire: str, netlist: dict, visited: set) -> int: # circ, wire: G4, netlist: {G1: 0, G2: 1, G3: 0, G4: -1, G5: -1}
   visited.add(wire)
   # print("GATE " + wire)
   if netlist[wire] != -1:
      return netlist[wire]
   else:
      gate = circ[wire][0]
      wire_in = circ[wire][1:]
      val_in = []
      # if wire in visited:
      #    return gateSim(gate, [netlist[w] for w in wire_in])
      for w in wire_in:
         if w in visited:
            print("END OF LINE FOR GATE " + w)
            return gateSim(gate, [netlist[i] for i in wire_in])
         # visited.add(w)
         val_in.append(simDFS(circ, w, netlist, visited))
      return gateSim(gate, val_in)



def gateSim(gate: str, inputs: list) -> int: # gate: 'OR', inputList: [1, 0]
   # if -1 in inputs:
      # raise Exception("ERROR: " + gate + " CONTAINS ONE OR MORE FLOATING INPUTS")
   if gate == 'NOT':
      return 1 - inputs[0]
   elif gate == 'DFF':
      return inputs[0]
   elif gate == 'AND':
      # if 0 in inputs:
      #    return 0
      return 0 + all(inputs)
   elif gate == 'NAND':
      # if 0 in inputs:
      #    return 1
      # else:
      #    return 0
      return 1 - all(inputs)
   elif gate == 'OR':
      return 0 + any(inputs)
   elif gate == 'NOR':
      return 1 - any(inputs)
   elif gate == 'XOR':
      return inputs.count(1) % 2
   elif gate == 'XNOR':
      return (inputs.count(1) + 1) % 2
   else:
      raise Exception("ERROR: GATE TYPE " + gate + " NOT SUPPORTED")


def getControl(benchfile: str) -> pd.DataFrame:
   circ, wire_in = createDict(benchfile)
   #  df = pd.read_table(testfile, delimiter='\t')

   # generate truth table
   tt_inputs = list(product([0, 1], repeat=len(wire_in)))

   tt = {index: [-1] for index in (wire_in + list(circ.keys()))}

   for line in tt_inputs:
      netlist = sim(line, circ, wire_in)
      for key, val in tt.items():
         val.append(netlist[key])

   df = pd.DataFrame(tt).drop(0).reset_index(drop=True)

   df.to_csv("truthtable.csv", index=False)

   # getting control values
   control_df = pd.DataFrame(
       np.zeros((len(circ), len(wire_in))), index=circ.keys(), columns=wire_in)
   control = 0

   for input in wire_in:
      df = df.sort_values(by=input).reset_index(drop=True)
      for key in circ:
         # print("control value of " + input + " on wire " + key + ":", end=' ')
         for index in range(len(df) // 2):
            control += df.loc[index, key] ^ df.loc[(index + len(df) // 2), key]
         # print(control)
         control_df.loc[key, input] = control / (2 ** len(input))
         control = 0

   return control_df


In [ ]:
circ, wire_in = createDict('testfiles/s298.bench')

control_df = pd.DataFrame(np.zeros((len(circ), len(wire_in))), index=circ.keys(), columns=wire_in)

rand = True
num_samples = 16384
if num_samples > (2 ** len(wire_in)):
    num_samples = 2 ** len(wire_in)
    rand = False

for index, wire in enumerate(wire_in):
    prev_combinations = []
    control_vals = {i: 0 for i in circ.keys()}
    comb0 = [0 for i in range(len(wire_in))]
    for i in range(num_samples):
        if rand:
            if i >= (num_samples // 2):
                break
            while comb0 in prev_combinations:
                # roll until new combination
                randint = np.random.randint(num_samples) # 42 # 2
                randint_bin = bin(randint).split('b')[1].zfill(len(wire_in))
                comb0 = [int(k) for k in randint_bin] #101010 -> [1, 0, 1, 0, 1, 0]  # [0, 0, 0, 0, 1, 0]
                comb0[index] = 0 # [0, 0, 1, 0, 1, 0]
            # append new combination to list
            prev_combinations.append(comb0)
        else:
            bin_string = bin(i).split('b')[1].zfill(len(wire_in))
            comb0 = [int(k) for k in bin_string]
            if comb0[index] == 1:
                continue
        comb1 = comb0.copy()
        netlist0 = sim(comb0, circ, wire_in)
        comb1[index] = 1
        netlist1 = sim(comb1, circ, wire_in)
        for key in control_vals:
            control_vals[key] += netlist0[key] ^ netlist1[key]
    # prev_strings = set([str(j) for j in prev_combinations])
    # if len(prev_strings) > len(prev_strings):
    #     print("DUPLICATES")
    control_df[wire] = control_vals.values()
    


control_df = control_df.div(num_samples)

print(control_df.to_string())

In [ ]:
control_df = control_df / num_samples

print(control_df.to_string())

In [ ]:
# circ, wire_in = createDict('testfiles/s27.bench')


# # generate truth table
# tt_inputs = list(product([0, 1], repeat=len(wire_in)))

# tt = {index: [-1] for index in (wire_in + list(circ.keys()))}

# for line in tt_inputs:
#       netlist = sim(line, circ, wire_in)
#       for key, val in tt.items():
#          val.append(netlist[key])

# df = pd.DataFrame(tt).drop(0).reset_index(drop=True)
# # print(df.to_string())


# # getting control values
# control_df = pd.DataFrame(np.zeros((len(circ), len(wire_in))), index=circ.keys(), columns=wire_in)
# control = 0
# for wire in wire_in:
#    df = df.sort_values(by=wire).reset_index(drop=True)
#    for key in circ:
#       # print("control value of " + input + " on wire " + key + ":", end=' ')
#       for index in range(len(df) // 2):
#          control += df.loc[index, key] ^ df.loc[(index + len(df) // 2), key]
#       # print(control)
#       control_df.loc[key, wire] = control # / (2 ** len(wire_in))
#       control = 0

# print(control_df.to_string())

In [3]:
# TEST VECTOR FILE GEN

num_inputs = 76
num_samples =  2 ** 12

if log2(num_samples) >= num_inputs:
    raise Exception("ERROR: NUM OF SAMPLES EXCEEDS POSSIBLE NUMBER OF INPUT COMBINATIONS")

vec = open("vectest12.vec", "w")
# vec.write(str(num_inputs) + '\n')

for i in range(num_inputs):
    prev_combinations = set()
    rand0 = '0'.zfill(num_inputs)
    for j in range(num_samples):
        while rand0 in prev_combinations:
            # roll until new combination
            rand_int = random.randint(0, 2 ** num_inputs)
            randint_bin = bin(rand_int).split('b')[1].zfill(num_inputs)
            rand0 = randint_bin[:i] + '0' + randint_bin[i + 1:]
        # append new combination to list
        prev_combinations.add(rand0)
        rand1 = rand0[:i] + '1' + rand0[i + 1:]
        vec.write(rand0 + '\n')
        vec.write(rand1 + '\n')
# vec.write("END")

vec.close()